In [157]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost.sklearn import XGBClassifier
from sklearn.grid_search import GridSearchCV

In [158]:
from utils.metrics import ndcg_scorer

In [166]:
path = '../data/processed/'
train_users = pd.read_csv(path + '_encoded_train_users.csv', nrows=5000)
train_users.fillna(-1, inplace=True)

y_train = train_users['country_destination']
train_users.drop('country_destination', axis=1, inplace=True)
train_users.drop('id', axis=1, inplace=True)

In [167]:
x_train = train_users.values
label_encoder = LabelEncoder()
encoded_y_train = label_encoder.fit_transform(y_train)

In [168]:
xgb = XGBClassifier()

In [126]:
xgb.fit(x_train, encoded_y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [127]:
preds = xgb.predict_proba(x_train)

In [128]:
preds[0]

array([ 0.00122057,  0.01080456,  0.00688097,  0.02602542,  0.03457563,
        0.01375806,  0.00878981,  0.40028012,  0.00304236,  0.00095095,
        0.46605584,  0.0276157 ], dtype=float32)

In [184]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

clf.fit(x_train, encoded_y_train)

rf_preds = clf.predict_proba(x_train)

In [139]:
ndcg_scorer(xgb, x_train, encoded_y_train)

0.78379958580567899

In [140]:
ndcg_scorer(clf, x_train, encoded_y_train)

0.98671378045645364

In [162]:
from sklearn.cross_validation import KFold

In [171]:
kf = KFold(len(x_train), n_folds=3)

In [149]:
a = []

In [153]:
a = []
for train_index, test_index in kf:
    X_train, X_test = x_train[train_index], x_train[test_index]
    y_train, y_test = encoded_y_train[train_index], encoded_y_train[test_index]
    clf = XGBClassifier()
    clf.fit(X_train, y_train)
    preds = clf.predict_proba(X_test)
    a.append(ndcg_score(y_test, preds))

In [151]:
np.mean(a)

0.7078027923779957

In [192]:
clf = GridSearchCV(
    clf,
    {
#         'max_depth': [4, 6, 8],
        'n_estimators': [4, 5, 6],
#         'learning_rate': [0.15, 0.2],
    },
    cv=kf,
    n_jobs=1,
    scoring=ndcg_scorer,
    pre
)

In [193]:
clf.fit(x_train, encoded_y_train)

ValueError: Invalid parameter n_estimators for estimator GridSearchCV. Check the list of available parameters with `estimator.get_params().keys()`.

In [187]:
clf.best_score_

0.7054394899415829